### FYI: Group 1 helped make most of this code. We wrote our own code in DataManipulation.ipynb but found that this gave us better results.

In [1]:
from gensim.test.utils import common_texts, get_tmpfile, datapath
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence, PathLineSentences
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors
from gensim.models.callbacks import CallbackAny2Vec
from itertools import combinations
import os.path
import numpy as np
from disjoint_set import DisjointSet
import pandas as pd

In [2]:
def get_top20(filename):
    file = open(filename, "r")
    top20 = file.read()

    removeChars = "[',]"

    for char in removeChars:
        top20 = top20.replace(char, "")

    top20 = top20.split()
    top20 = [opcode.lower() for opcode in top20]
    return top20

In [3]:
class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 1

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [4]:
args = {'family': "CeeInject"}

base_path = './data/'

path = base_path + args['family']

In [5]:
output_fname = args['family'] + "_output.txt"
f = open(output_fname,"w+")
f.write(args['family'] + "\n")

10

In [6]:
data = PathLineSentences(path, limit=50)
top20 = get_top20(base_path + "top20.txt")

## 1AB

In [7]:
model = Word2Vec(data, size=2, window=6, min_count=1, \
                 workers=4)
model_file = args['family'] + ".model"
model.save(model_file)

word_vectors = model.wv
opcode_vectors = []

f.write("Opcode Pairings:\n")
for i, opcode in enumerate(word_vectors.vocab):
    opcode_vectors.append(word_vectors[opcode])
    f.write(f"{opcode}: " +  str(opcode_vectors[i]) + "\n")

f.write("\n"*5)

5

In [8]:
similarity = pd.DataFrame(index=top20, columns=top20)
opcode_pairs = combinations(top20, 2)
for x, y in combinations(top20, 2):
    similarity[x][y] = similarity[y][x] = word_vectors.similarity(x, y)
similarity.fillna(1.0, inplace=True)

In [9]:
similarity

,push,mov,inc,dec,pop,add,xor,xchg,or,adc,sub,and,sbb,cmp,imul,out,outsl,jo,js,insl
push,1.000000,0.087611,0.771003,-0.564516,0.550756,0.536107,-0.566369,-0.175598,0.948766,-0.991943,-0.352256,0.882847,0.352978,0.741127,0.751893,0.785292,-0.815340,-0.646200,0.548192,0.323915
mov,0.087611,1.000000,0.701931,0.772790,0.879709,0.887873,0.771363,-0.996061,0.397888,-0.213104,-0.963166,0.545202,0.962959,-0.603852,0.722624,0.685545,0.505323,-0.816848,0.881164,0.970827
inc,0.771003,0.701931,1.000000,0.090410,0.956176,0.950921,0.088174,-0.762322,0.932728,-0.845469,-0.867603,0.979773,0.867986,0.143866,0.999565,0.999741,-0.259916,-0.984232,0.955273,0.852237
dec,-0.564516,0.772790,0.090410,1.000000,0.378041,0.394139,0.999997,-0.713469,-0.274776,0.455399,-0.573661,-0.110713,0.573029,-0.972538,0.119732,0.067730,0.938178,-0.265145,0.380880,0.598065
pop,0.550756,0.879709,0.956176,0.378041,1.000000,0.999848,0.375961,-0.918409,0.786277,-0.652059,-0.975174,0.878244,0.975345,-0.152185,0.964393,0.949268,0.034203,-0.992889,0.999995,0.968064
add,0.536107,0.887873,0.950921,0.394139,0.999848,1.000000,0.392074,-0.925173,0.775375,-0.638729,-0.978890,0.869765,0.979047,-0.169409,0.959630,0.943635,0.051639,-0.990661,0.999897,0.972292
xor,-0.566369,0.771363,0.088174,0.999997,0.375961,0.392074,1.000000,-0.711893,-0.276935,0.457397,-0.571820,-0.112945,0.571187,-0.973058,0.117502,0.065489,0.938953,-0.262979,0.378802,0.596264
xchg,-0.175598,-0.996061,-0.762322,-0.713469,-0.918409,-0.925173,-0.711893,1.000000,-0.477672,0.298901,0.983217,-0.617389,-0.983076,0.530793,-0.781071,-0.747401,-0.426814,0.864781,-0.919618,-0.988265
or,0.948766,0.397888,0.932728,-0.274776,0.786277,0.775375,-0.276935,-0.477672,1.000000,-0.981152,-0.629936,0.986019,0.630535,0.491018,0.921692,0.940690,-0.590619,-0.854239,0.784377,0.606264
adc,-0.991943,-0.213104,-0.845469,0.455399,-0.652059,-0.638729,0.457397,0.298901,-0.981152,1.000000,0.467984,-0.935233,-0.468665,-0.650103,-0.829358,-0.857399,0.735422,0.737676,-0.649730,-0.441161


In [10]:
f.write("Similarity Scores\n\n")
for idx, value in similarity.stack().iteritems():
    f.write('{:>5} <> {:<6}: {:7.4f}\n'.format(idx[0], idx[1], value))
f.write("\n"*5)

5

In [11]:
ds = DisjointSet()
    
for i, j in opcode_pairs:
    ds.find(i)
    ds.find(j)
    if similarity[i][j] > 0.90:
        ds.union(i, j)
        
sets = list(ds.itersets())
f.write("Pairings:\n\n")
for s in sets:
    f.write(str(s) + "\n")
f.write("\n"*5)

5

In [12]:
f.close()

### 1C

In [ ]:
path = base_path + args['family']

In [13]:
samples = []
models = []

for i, file in enumerate(sorted(os.listdir(path))):
    print("file:", file)
    samples.append(LineSentence(path + "/" + file, limit=50))
    models.append(Word2Vec(samples[i], size=2, window=6, min_count=1,
                           workers=4))

file: 100548.txt
file: 100817.txt
file: 100982.txt
file: 102435.txt
file: 105104.txt
file: 105713.txt
file: 105985.txt
file: 107859.txt
file: 107964.txt
file: 108310.txt
file: 108654.txt
file: 109332.txt
file: 110257.txt
file: 112137.txt
file: 112788.txt
file: 115690.txt
file: 116878.txt
file: 118861.txt
file: 119561.txt
file: 119899.txt
file: 120493.txt
file: 121234.txt
file: 121813.txt
file: 124004.txt
file: 126061.txt
file: 127582.txt
file: 129324.txt
file: 130343.txt
file: 133338.txt
file: 136636.txt
file: 137025.txt
file: 137911.txt
file: 138247.txt
file: 138425.txt
file: 139386.txt
file: 140005.txt
file: 142047.txt
file: 143637.txt
file: 143785.txt
file: 144857.txt
file: 145040.txt
file: 146900.txt
file: 147061.txt
file: 147343.txt
file: 147809.txt
file: 148690.txt
file: 150130.txt
file: 150216.txt
file: 150470.txt
file: 150961.txt
file: 151525.txt
file: 151632.txt
file: 152656.txt
file: 152838.txt
file: 152847.txt
file: 154164.txt
file: 154505.txt
file: 155525.txt
file: 156546.t

file: 569572.txt
file: 573582.txt
file: 574455.txt
file: 575335.txt
file: 575606.txt
file: 575759.txt
file: 575870.txt
file: 577195.txt
file: 579109.txt
file: 580006.txt
file: 581757.txt
file: 582190.txt
file: 583412.txt
file: 584480.txt
file: 584858.txt
file: 585409.txt
file: 586351.txt
file: 586921.txt
file: 588011.txt
file: 589360.txt
file: 590279.txt
file: 592065.txt
file: 592281.txt
file: 595353.txt
file: 597135.txt
file: 597785.txt
file: 600052.txt
file: 600204.txt
file: 601097.txt
file: 601270.txt
file: 601391.txt
file: 601743.txt
file: 602061.txt
file: 602589.txt
file: 603567.txt
file: 604772.txt
file: 605817.txt
file: 605818.txt
file: 606631.txt
file: 607919.txt
file: 608060.txt
file: 608623.txt
file: 613313.txt
file: 615397.txt
file: 615920.txt
file: 620401.txt
file: 620522.txt
file: 621477.txt
file: 628877.txt
file: 630343.txt
file: 630366.txt
file: 631087.txt
file: 631110.txt
file: 631281.txt
file: 636451.txt
file: 636706.txt
file: 637323.txt
file: 637338.txt
file: 637361.t

In [14]:
embedding_vectors = np.array([])

top20 = get_top20(base_path + "top20.txt")

for m in models:
    for i in range(20):
        if top20[i] not in m.wv.vocab:
            embedding_vectors = np.append(embedding_vectors, [0, 0])
        else:
            embedding_vectors = np.append(embedding_vectors, m.wv[top20[i]])
embedding_vectors.shape

(36000,)

In [15]:
embedding_vectors = np.reshape(embedding_vectors, (40, 900))
embedding_vectors = np.transpose(embedding_vectors)

In [16]:
f = open('embedding_vectors', 'a+')
f.write(args['family'] + " EMBEDDING VECTORS -----------------------------------------\n")
for vec in embedding_vectors:
    f.write(str(vec.tolist()) + "\n")